# Lesson 8: Dynamic Contexts

🟡 Intermediate · ⏱ 20 min

---

Contexts are background key-value pairs that get injected into every agent request — silently, automatically. Use them for user profiles, company information, current date/time, environment settings, or anything the agent should always know.

Unlike **memory** (which stores conversation history), contexts are **static facts you control** — you create them, update them, and attach them to agents.

## Objectives

By the end of this lesson you will be able to:

1. Explain the difference between contexts and memory
2. Create, update, and delete contexts
3. Attach and detach contexts from agents
4. Use multiple contexts simultaneously
5. Update contexts dynamically without recreating agents

## Prerequisites

Before starting this lesson, make sure you have completed:

- **Lesson 1** — Studio setup and `agent.run()`
- **Lesson 2** — Providers and models
- **Lesson 3** — Agent lifecycle (create, update, delete)
- **Lesson 4** — Structured outputs
- **Lesson 5** — Memory and sessions
- **Lesson 6** — Tools and functions
- **Lesson 7** — Knowledge bases and RAG

You also need:

- A valid `LYZR_API_KEY` set as an environment variable (or ready to paste below)

In [ ]:
!pip install lyzr-adk -q

In [ ]:
import os
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## Contexts vs Memory

These two features are easy to confuse at first. Here is a clear side-by-side comparison:

| | Contexts | Memory |
|---|---|---|
| **What it stores** | Background facts you provide | Conversation history |
| **Who controls it** | You, via the SDK | The agent automatically |
| **When it applies** | Every single request | Depends on session |
| **Use case** | User profile, company info, feature flags | Multi-turn conversations |
| **API** | `create_context` + `add_context` | `add_memory` |
| **Mutability** | You update it explicitly | Grows automatically with conversation |
| **Scope** | Any agent you attach it to | Tied to the session |

**Rule of thumb:** If it is information *you* know and want the agent to always have — use a context. If it is information that emerges from the *conversation* — that is memory.

## Creating Your First Context

A context is a named key-value pair stored in your Lyzr account. You create it once and can reuse it across multiple agents.

`studio.create_context(name, value)` returns a context object with an `id` you can use to retrieve or manage it later.

In [ ]:
# Create a user profile context
user_ctx = studio.create_context(
    name="user_profile",
    value="User: Alice Chen, Role: Senior Software Engineer, Team: Platform, Timezone: UTC+8, Preferred language: Python"
)
print(f"Context created!")
print(f"  Name:  {user_ctx.name}")
print(f"  ID:    {user_ctx.id}")
print(f"  Value: {user_ctx.value if hasattr(user_ctx, 'value') else '(stored)'}")

## Attaching a Context to an Agent

Once you have a context, attach it to an agent with `agent.add_context(ctx)`. From that point on, the context value is **silently injected with every request** — you do not need to mention Alice's profile in your messages at all.

This is powerful: you write generic prompts, and the agent personalizes its responses automatically.

In [ ]:
agent = studio.create_agent(
    name="Personalized Assistant",
    provider="openai/gpt-4o",
    role="Personal productivity assistant",
    goal="Give personalized, relevant advice",
    instructions="Always take the user's profile into account when answering. Personalize every response."
)

agent.add_context(user_ctx)
print("Context attached to agent!")

# The agent now knows Alice's profile without being told in the message
r1 = agent.run("What timezone should I schedule a meeting for my team in San Francisco?")
print(f"\nTimezone question: {r1.response}")

r2 = agent.run("Recommend a way to improve my code review process.")
print(f"\nCode review question: {r2.response}")

## Updating a Context

Contexts are mutable. Call `ctx.update(new_value)` to change the value. All subsequent agent requests — on any agent that has this context attached — will immediately use the updated value.

You do **not** need to recreate the agent or detach/reattach the context. The update propagates automatically.

In [ ]:
# Alice got promoted!
user_ctx.update(
    "User: Alice Chen, Role: Engineering Manager, Team: Platform, Timezone: UTC+8, Direct reports: 6"
)
print("Context updated — Alice is now Engineering Manager.")

# Agent immediately uses the updated context
r3 = agent.run("What are my main priorities this week?")
print(f"\nPriorities (new role): {r3.response}")

## Multiple Contexts

You can attach as many contexts as needed to a single agent. Each context becomes part of the agent's background knowledge. A common pattern is to have separate contexts for:

- **User profile** — who the person is
- **Company info** — organizational context
- **Current project** — what they are working on
- **Feature flags / env settings** — runtime configuration

Keeping contexts separate makes it easy to update one without touching the others.

In [ ]:
# Add company context
company_ctx = studio.create_context(
    name="company_info",
    value="Company: TechCorp, Industry: B2B SaaS, Stage: Series B, Size: 150 employees, Main product: DevOps platform"
)

# Add current project context
project_ctx = studio.create_context(
    name="current_project",
    value="Project: API Gateway Migration, Deadline: Q2 2026, Stack: Python/FastAPI/AWS, Status: In progress (40% done)"
)

agent.add_context(company_ctx)
agent.add_context(project_ctx)
print("2 more contexts added!")

r4 = agent.run("Give me a brief status update I could send to my CTO about the current project.")
print(f"\nStatus update: {r4.response}")

## Removing a Context

When a context is no longer relevant, detach it from the agent with `agent.remove_context(ctx)`. The agent will stop receiving that background information on all future requests.

Note: `remove_context` detaches the context from the agent — it does not delete the context object from your account. To permanently delete a context, use `ctx.delete()`.

In [ ]:
# Remove the project context (project is done)
agent.remove_context(project_ctx)
print("Project context removed.")

r5 = agent.run("What project am I currently working on?")
print(f"\nAfter removing project context: {r5.response}")
# Agent won't know about the API Gateway project anymore

## Listing All Contexts

`studio.list_contexts()` returns all contexts in your account — useful for auditing what background information you have defined, or for retrieving a context by name without storing its reference.

In [ ]:
all_contexts = studio.list_contexts()
print(f"You have {len(all_contexts)} context(s):")
for ctx in all_contexts:
    print(f"  • {ctx.name} (ID: {ctx.id})")

## Common Mistake: Contexts vs System Prompts

A frequent mistake is hardcoding dynamic information directly into the agent's `instructions` (system prompt). This looks convenient at first, but creates two problems:

1. **Hard to update** — changing the instructions requires calling `agent.update()` and rebuilding the agent configuration
2. **Not reusable** — you would have to duplicate the same information in every agent that needs it

Use the **instructions** for stable, role-defining behavior that rarely changes. Use **contexts** for dynamic, per-user, or per-session information that you want to swap easily.

In [ ]:
# Antipattern: hardcoding user info in instructions
hardcoded_agent = studio.create_agent(
    name="Hardcoded Agent",
    provider="openai/gpt-4o",
    role="Assistant",
    goal="Help users",
    instructions="The user's name is Alice and she is an engineer."  # Hard to update
)

# Better: use contexts for dynamic/changing info
dynamic_agent = studio.create_agent(
    name="Dynamic Agent",
    provider="openai/gpt-4o",
    role="Assistant",
    goal="Help users",
    instructions="Use the user profile context to personalize responses."  # Flexible
)
dynamic_agent.add_context(user_ctx)  # Easy to swap or update

print("Contexts make agent behavior dynamic without recreating agents.")

## Exercise: Cooking Assistant with Multiple Contexts

Build a cooking assistant agent that uses three separate contexts:

1. **dietary_preferences** — e.g. vegetarian, allergies, flavor preferences
2. **kitchen_equipment** — what appliances and tools the user has
3. **cooking_skill** — experience level and time available

Then:
- Ask for a dinner recommendation
- Update one context (e.g. change dietary preferences)
- Ask again and observe how the recommendation changes

Fill in the `...` placeholders below.

In [ ]:
# TODO: Create 3 contexts for a cooking assistant
dietary_ctx = studio.create_context(
    name="dietary_preferences",
    value=...  # e.g. "Vegetarian, nut allergy, loves spicy food"
)

equipment_ctx = studio.create_context(
    name="kitchen_equipment",
    value=...  # e.g. "Has: oven, stovetop, blender, air fryer. No: grill"
)

skill_ctx = studio.create_context(
    name="cooking_skill",
    value=...  # e.g. "Beginner cook, comfortable with 30-minute meals"
)

# TODO: Create a cooking assistant agent
chef_agent = studio.create_agent(
    name=...,
    provider="openai/gpt-4o",
    role=...,
    goal=...,
    instructions=...
)

# TODO: Add all three contexts
chef_agent.add_context(...)
chef_agent.add_context(...)
chef_agent.add_context(...)

# TODO: Ask for a recipe recommendation
response = chef_agent.run("What should I make for dinner tonight?")
print(response.response)

# TODO: Update one context and see how the recommendation changes
dietary_ctx.update(...)
response2 = chef_agent.run("What should I make for dinner tonight?")
print(response2.response)

## Summary

### Contexts vs Memory — Quick Recap

| | Contexts | Memory |
|---|---|---|
| **What it stores** | Background facts you provide | Conversation history |
| **Who controls it** | You, via the SDK | The agent automatically |
| **When it applies** | Every single request | Depends on session |
| **Use case** | User profile, company info | Multi-turn conversations |

### API Quick Reference

| Operation | Code |
|---|---|
| Create a context | `ctx = studio.create_context(name="...", value="...")` |
| Retrieve by ID | `ctx = studio.get_context(ctx_id)` |
| List all contexts | `all_ctxs = studio.list_contexts()` |
| Update value | `ctx.update("new value")` |
| Delete context | `ctx.delete()` |
| Attach to agent | `agent.add_context(ctx)` |
| Detach from agent | `agent.remove_context(ctx)` |

### Key Takeaways

- **Contexts are reusable** — create once, attach to many agents
- **Contexts are updatable** — change the value and all attached agents immediately use the new value, no agent rebuild needed
- **Contexts are composable** — attach multiple contexts to a single agent for layered background knowledge
- **Best for static/semi-static facts** — user profile, company info, current date, environment flags
- **Not a replacement for instructions** — use instructions for stable, role-defining behavior; use contexts for dynamic per-user or per-session data

## Next Steps

**Lesson 9: Responsible AI Guardrails**

Now that your agents are personalized with contexts, learn how to keep them safe and compliant. Lesson 9 covers the Governor service — input/output filtering, content policies, and responsible AI guardrails you can configure programmatically.

---

*lyzr-adk series — Lesson 8 of 10*